In [35]:
import requests
import json
import numpy as np
import pandas as pd

# Hide errors while testing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

DISP_IDS = {
  "flowertopia": "614cdcbdea191200a8a1ed2f",
  "toke" : "5f8758560d683700eab65918",
  "Shiva's rose":"5fdd352ebb29a301034435d5",
  "Growers Retail": "5eb9a5221c1db800f8bd2a7d"
}

COLS = [
    'seller',
    'cName', 
    'name',
    'brandName',
    'Prices',
    'recOnly',
    'recPrices',
    'special', 
    'specialData', 
    'type', 
    'wholesalePrices'
]

def getProducts(id):
  x = requests.get(f'https://dutchie.com/graphql?operationName=FilteredProducts&variables=%7B%22includeEnterpriseSpecials%22%3Afalse%2C%22includeCannabinoids%22%3Atrue%2C%22productsFilter%22%3A%7B%22dispensaryId%22%3A%22{id}%22%2C%22pricingType%22%3A%22rec%22%2C%22strainTypes%22%3A%5B%5D%2C%22subcategories%22%3A%5B%5D%2C%22Status%22%3A%22Active%22%2C%22types%22%3A%5B%5D%2C%22useCache%22%3Afalse%2C%22sortDirection%22%3A1%2C%22sortBy%22%3Anull%2C%22isDefaultSort%22%3Atrue%2C%22bypassOnlineThresholds%22%3Afalse%2C%22isKioskMenu%22%3Afalse%2C%22removeProductsBelowOptionThresholds%22%3Atrue%7D%2C%22page%22%3A0%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22699c79a80fc45daf41c34407247fd0c38dd698fd595d36501b90a3104931ef1a%22%7D%7D').text
  clean = json.loads(x)
  prodList = clean['data']['filteredProducts']['products']
  return prodList

def parsePrice(priceList, name):
  if len(priceList) == 0:
    return None
  else:
    if len(priceList) > 1:
      print('Product' + name + 'has more than one price listed, using the first one listed')
  return priceList[0]

def toDataFrame(prodList, seller):
  infoDataFrame = pd.DataFrame(columns=COLS)
  for prod in prodList:
    price = parsePrice(prod['Prices'], prod['cName'])
    recSpecialPrices = parsePrice(prod['recSpecialPrices'], prod['cName'])

      
    row = pd.DataFrame([[seller,prod['cName'], prod['Name'], prod['brandName'], price, prod['recOnly'],recSpecialPrices, prod['special'], prod['specialData'], prod['type'], prod['wholesalePrices']]],columns=COLS, index=['1'])
    infoDataFrame = infoDataFrame.append(row, ignore_index=True)
  return infoDataFrame


competitorsDF = pd.DataFrame(columns=COLS)

flowertpiaDF = pd.DataFrame(columns=COLS)

for disp in DISP_IDS:
  id = DISP_IDS[disp]
  prods = getProducts(id)
  df = toDataFrame(prods, disp)
  df.to_csv(path_or_buf=f'./out/{disp}.csv')
  if disp == 'flowertopia':
    flowertpiaDF = flowertpiaDF.append(df, ignore_index=True)
  else:
    competitorsDF = competitorsDF.append(df, ignore_index=True)

competitorsDF.to_csv(path_or_buf=f'./out/competitors.csv')




Productpink-kush-81105has more than one price listed, using the first one listed
Productpink-kushhas more than one price listed, using the first one listed
Productback-forty-animal-mintshas more than one price listed, using the first one listed
Productback-forty-fruity-pebbles-oghas more than one price listed, using the first one listed
Productback-forty-wedding-piehas more than one price listed, using the first one listed
Productedison-black-cherry-punchhas more than one price listed, using the first one listed
Productgood-supply-jean-guyhas more than one price listed, using the first one listed
Productgood-supply-monkey-glue-7694has more than one price listed, using the first one listed
Productgood-supply-pineapple-express-510-thread-cartridgehas more than one price listed, using the first one listed
Productgood-supply-purple-monkey-510-thread-cartridgehas more than one price listed, using the first one listed
Productkolab-project-kaliforniahas more than one price listed, using the f

In [67]:
flowertopiaProducts = flowertpiaDF['cName'].to_numpy()

comparissonColumns = [ "product", "flowertopia","toke", "Shiva's rose","Growers Retail"]

comparisonDF = pd.DataFrame(columns=comparissonColumns)




for prod in flowertopiaProducts:
  matches = competitorsDF.loc[competitorsDF['cName'] == prod]
  if len(matches) > 0:
    flowertopiaPrice = flowertpiaDF.loc[flowertpiaDF['cName'] == prod]['Prices'].values
    tokePrice = matches.loc[matches['seller'] == 'toke']['Prices'].values
    shivaPrice = matches.loc[matches['seller'] == "Shiva's rose"]['Prices'].values
    growersPrice = matches.loc[matches['seller'] == "Growers Retail"]['Prices'].values
    print(prod,flowertopiaPrice, tokePrice, shivaPrice, growersPrice)


    row = pd.DataFrame([[prod,flowertopiaPrice, tokePrice, shivaPrice, growersPrice]],columns=comparissonColumns, index=['1'])
    comparisonDF = comparisonDF.append(row, ignore_index=True)

comparisonDF.to_csv(path_or_buf=f'./out/comparisons.csv')

0-10-cbd-gems-15-pack [16] [13.27] [] []
8-ball-kush-shatter-double-infused-pre-roll-1g [16] [] [14.75] []
balanced-oil-30ml [35.95] [35.39] [] []
cbd-milk-chocolate [4.49] [5.31] [] []
chocolate-brownies [9.49] [] [11.98] []
cookies-and-cream-white-chocolate [4.99] [5.31] [] []
dissolvable-thc-powder-single-10mg [7.5] [7.96] [] []
funk-master [31] [30.97] [26.75] []
gnarberry [31] [30.97] [] []
grower-s-choice-indica-pre-roll-1g [6.99] [] [5.95] []
hazy-honey-0-5g [29.95] [42.48] [] []
japanese-citrus-yuzu-2-1-sour-soft-chews [7.99] [7.96] [7.5] [7.49]
jean-guy [17] [141.59] [] []
jean-guy-pre-roll-1g [8.2] [7.96] [] []
legacy-gorilla-cookies [18] [] [16.5] []
mandarin-cookie-pre-roll-3-pack-1-5g [18.3] [] [17.5] []
mango-sour-soft-chews [6.99] [7.96] [] []
pink-kush-pre-roll-10-pack-3g [20] [22.12] [] []
pink-kush-pre-roll-3-pack-1-5g [11.5] [10.62] [] []
raspberry-cheesecake-1-1-cookies-2-pack [9.4] [] [9.5] []
real-fruit-raspberry-soft-chew-2-pack [6.99] [7.97] [] []
reign-drops-30